In [37]:
# Author: Naveen Lalwani
# Script to train and Quantize baseline model LeNet-300-100 on CIFAR-10 dataset

import numpy as np
import tensorflow as tf
import keras
from collections import Counter
from tensorflow.contrib import lite
from keras.utils import np_utils
from keras import layers
from sklearn.utils import shuffle

<B> Network Parameters </B>

In [38]:
batch_size = 64
num_classes = 10
epochs = 500
learning_rate = 0.001
display_step = 50
n_input = 3072

<B> Loading CIFAR 10 Dataset and Preprocessing it </B>

In [39]:
(x_train, y_train), (x_test, y_test) =  tf.keras.datasets.cifar10.load_data()

# Enabling One Hot Encoding
y_train = np_utils.to_categorical(y_train, num_classes)
y_test = np_utils.to_categorical(y_test, num_classes)

# Changing input image datatype to float
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Normalizaig data
x_train  /= 255
x_test /= 255

x_train = x_train.reshape([50000, 3072])
x_test = x_test.reshape([10000, 3072])

In [40]:
# Placeholders
X = tf.placeholder(tf.float32, shape = [None, 3072], name = "X") # Placeholder for Images 32 X 32 size with 3 RGB channels
Y = tf.placeholder(tf.float32, shape = [None, 10], name = "Y") # Placeholder for Labels

In [41]:
weights = {
    # Fully Connected Layer 1: 3072 input channels, 300 output channels
    'w1' : tf.Variable(tf.random_normal([3072, 300]), name = 'w1'),
    # Fully Connected Layer 2: 300 input channels, 100 output channels
    'w2' : tf.Variable(tf.random_normal([300, 100]), name = 'w2'),
    # Fully Connected Layer 3: 100 input channels, 10 (number of classes) output channels
    'w3' : tf.Variable(tf.random_normal([100, 10]), name = 'w3')
}

biases = {
    'b1' : tf.Variable(tf.random_normal([300]), name = 'b1'),
    'b2' : tf.Variable(tf.random_normal([100]), name = 'b2'),
    'b3' : tf.Variable(tf.random_normal([10]), name = 'b3')
}

In [42]:
def lenet_300_100(x, weight, bias):
    b_min1 = tf.reduce_min(bias['b1'])
    b_max1 = tf.reduce_max(bias['b1'])
    b_fake_quant1 = tf.fake_quant_with_min_max_vars(bias['b1'], 
                    min=b_min1, 
                    max=b_max1, 
                    narrow_range=True, 
                    name="b_weights1")
    
    
    b_min2 = tf.reduce_min(bias['b2'])
    b_max2 = tf.reduce_max(bias['b2'])
    b_fake_quant2 = tf.fake_quant_with_min_max_vars(bias['b2'], 
                    min=b_min2, 
                    max=b_max2, 
                    narrow_range=True,
                    name="b_weights2")
    
    
    b_min3 = tf.reduce_min(bias['b3'])
    b_max3 = tf.reduce_max(bias['b3'])
    b_fake_quant3 = tf.fake_quant_with_min_max_vars(bias['b3'], 
                    min=b_min3, 
                    max=b_max3, 
                    narrow_range=True,
                    name="b_weights3")
    
    # Fully Connected Layer 1
    # First multiply the weights with inputs, then add the bias, then apply RELU activation
    w_min1 = tf.reduce_min(weight['w1'])
    w_max1 = tf.reduce_max(weight['w1'])
    w_fake_quant1 = tf.fake_quant_with_min_max_vars(weight['w1'], 
                    min=w_min1, 
                    max=w_max1, 
                    narrow_range=True, 
                    name="quant_weights1")
    
    # Fully Connected Layer 1
    # First multiply the weights with inputs, then add the bias, then apply RELU activation
    fc1 = tf.add(tf.matmul(x, weight['w1']), bias['b1']) # Linear Function
    fc1 = tf.nn.relu(fc1) # Activation Function
    
    # Fully Connected Layer 2
    fc2 = tf.add(tf.matmul(fc1, weight['w2']), bias['b2']) # Linear Function
    fc2 = tf.nn.relu(fc2) # Activation Function
    
    # Ouput Layer
    out = tf.add(tf.matmul(fc2, weight['w3']), bias['b3']) # Output Layer
    
    return out

<B> Evaluating Model Performance </B>

In [43]:
# Get probabilities for the input for all the classes
logits = lenet_300_100(X, weights, biases)

cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits = logits, labels = Y)
loss_op = tf.reduce_mean(cross_entropy)

# Using ADAM optimization
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate)
train_op = optimizer.minimize(loss_op)

# Get correct prediction by getting class with maximum probability and get accuracy
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))

# This calculates the classification accuracy by first type-casting the vector of booleans to floats, so that False becomes 0 and True 
# becomes 1, and then calculating the average of these numbers.
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [44]:
init = tf.global_variables_initializer()

In [45]:
# Set to use GPU for training Convolution layers and allow the memory to grow so that tensors can be loaded.
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

with tf.Session(config=config) as sess:
    sess.run(init)
    num_examples = 50000
    acc_hist = []
    cost_hist = []
    batch_x = x_train
    batch_y = y_train
    print("Training------------------")
    for i in range(1, epochs + 1):
        x_train, y_train = shuffle(x_train, y_train)
        for offset in range(0, num_examples, batch_size):
            end = offset + batch_size
            batch_x, batch_y = x_train[offset:end], y_train[offset:end]
            sess.run(train_op, feed_dict={X: batch_x, Y: batch_y}) 
        if (i % display_step == 0):
            loss, acc = sess.run([train_op, accuracy], feed_dict = {X: batch_x, Y: batch_y})
            loss = loss
            cost_hist.append(loss)
            acc_hist.append(acc)
            print('Epoch ' + str(i) + ', Cost: ' + str(loss) + ', Accuracy on batch: ' + str(acc * 100) + ' %')
            print("Test Accuracy: ", str(accuracy.eval({ X : x_test, Y : y_test}) * 100) + ' %')
    print('-' * 70)
    print('\nOptimization Finished\n')
    print('Now testing accuracy on the complete data, we have:\n')
    acc1 = 0
    for i in range(0, num_examples, 5000):
        end = i + 5000
        acc1 = acc1 + accuracy.eval({ X : x_train[i:end], Y : y_train[i:end]})
    print("Train Accuracy: ", str((acc1 / 10) * 100) + ' %')
    print("Test Accuracy: ", str(accuracy.eval({ X : x_test, Y : y_test}) * 100) + ' %')
    
    # Saving the full precision model
    converter = lite.TFLiteConverter.from_session(sess, [X], [logits])
    tflite_model = converter.convert()
    open("LeNet_float32_model_CIFAR-10.tflite", "wb").write(tflite_model)
    
    # Saving the quantized model
    converter = lite.TFLiteConverter.from_session(sess, [X], [logits])
    converter.inference_input_type = tf.float32
    converter.default_ranges_stats = (0., 6.)
    input_mean = 128
    input_stddev = 128
    input_arrays = converter.get_input_arrays()
    converter.quantized_input_stats = {input_arrays[0] : (input_mean, input_stddev)}
    converter.post_training_quantize = True
    tflite_model = converter.convert()
    open("LeNet_int8_model_CIFAR-10.tflite", "wb").write(tflite_model)

Training------------------
Epoch 50, Cost: None, Accuracy on batch: 43.75 %
Test Accuracy:  34.72000062465668 %
Epoch 100, Cost: None, Accuracy on batch: 81.25 %
Test Accuracy:  38.71999979019165 %
Epoch 150, Cost: None, Accuracy on batch: 56.25 %
Test Accuracy:  40.16999900341034 %
Epoch 200, Cost: None, Accuracy on batch: 93.75 %
Test Accuracy:  44.67000067234039 %
Epoch 250, Cost: None, Accuracy on batch: 62.5 %
Test Accuracy:  45.089998841285706 %
Epoch 300, Cost: None, Accuracy on batch: 68.75 %
Test Accuracy:  44.5499986410141 %
Epoch 350, Cost: None, Accuracy on batch: 81.25 %
Test Accuracy:  44.200000166893005 %
Epoch 400, Cost: None, Accuracy on batch: 93.75 %
Test Accuracy:  43.57999861240387 %
Epoch 450, Cost: None, Accuracy on batch: 87.5 %
Test Accuracy:  43.43000054359436 %
Epoch 500, Cost: None, Accuracy on batch: 93.75 %
Test Accuracy:  43.5699999332428 %
----------------------------------------------------------------------

Optimization Finished

Now testing accuracy 